In [1]:
from sympy import *
import matplotlib.pyplot as plt

init_printing()

In [2]:
c = [Matrix(x) for x in [(-1, 1), ( 0, 1), ( 1, 1), (-1, 0), ( 0, 0), ( 1, 0), (-1,-1), ( 0, -1), ( 1, -1)]]

In [3]:
c

⎡⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0⎤  ⎡1⎤  ⎡-1⎤  ⎡0 ⎤  ⎡1 ⎤⎤
⎢⎢  ⎥, ⎢ ⎥, ⎢ ⎥, ⎢  ⎥, ⎢ ⎥, ⎢ ⎥, ⎢  ⎥, ⎢  ⎥, ⎢  ⎥⎥
⎣⎣1 ⎦  ⎣1⎦  ⎣1⎦  ⎣0 ⎦  ⎣0⎦  ⎣0⎦  ⎣-1⎦  ⎣-1⎦  ⎣-1⎦⎦

In [4]:
w = [Rational(*x) for x in [(1,36), (1,9), (1,36), (1,9), (4,9), (1,9), (1,36), (1,9), (1,36)]]

In [5]:
w

In [6]:
sum(w)

In [7]:
c_s = sqrt(Rational(1,3))
c_s

In [22]:
u_x, u_y, rho, tau = symbols('u_x u_y rho tau')

In [23]:
f_next = symarray('f_next', 9)
f_next

array([f_next_0, f_next_1, f_next_2, f_next_3, f_next_4, f_next_5,
       f_next_6, f_next_7, f_next_8], dtype=object)

In [24]:
f_curr = symarray('f_curr', 9)
f_curr

array([f_curr_0, f_curr_1, f_curr_2, f_curr_3, f_curr_4, f_curr_5,
       f_curr_6, f_curr_7, f_curr_8], dtype=object)

In [25]:
rho = sum(f_curr)
rho

In [55]:
#u_x = sum([ (c_i*f_curr[i])[0] for i, c_i in enumerate(c) ]) / rho
#u_x

In [56]:
#u_y = sum([ (c_i*f_curr[i])[1] for i, c_i in enumerate(c) ]) / rho
#u_y

In [26]:
u = Matrix([u_x, u_y])
u

⎡uₓ ⎤
⎢   ⎥
⎣u_y⎦

In [27]:
f_eq = []

for i, c_i in enumerate(c):
    f_eq_i = w[i] * rho * (  1
                           + c_i.dot(u)    /    c_s**2
                           + c_i.dot(u)**2 / (2*c_s**4)
                           - u.dot(u)      / (2*c_s**2) )
    f_eq.append(f_eq_i)

f_eq

In [28]:
f_next = [ f_curr[i] + 1/tau * ( f_eq_i - f_curr[i] ) for i, f_eq_i in enumerate(f_eq) ]
f_next

In [29]:
cse(f_next)

In [30]:
eqs = cse(f_next, optimizations='basic')
eqs

In [31]:
for eq in eqs[0]:
    print("const float %s = %s;" % (eq[0], ccode(eq[1])))

const float x0 = f_curr_0 + f_curr_1 + f_curr_2 + f_curr_3 + f_curr_4 + f_curr_5 + f_curr_6 + f_curr_7 + f_curr_8;
const float x1 = 6*u_y;
const float x2 = 6*u_x;
const float x3 = pow(u_y, 2);
const float x4 = 3*x3;
const float x5 = pow(u_x, 2);
const float x6 = 3*x5;
const float x7 = x6 - 2;
const float x8 = x4 + x7;
const float x9 = x2 + x8;
const float x10 = 1.0/tau;
const float x11 = (1.0/72.0)*x10;
const float x12 = 6*x3;
const float x13 = x1 - x6 + 2;
const float x14 = (1.0/18.0)*x10;
const float x15 = -x4;
const float x16 = 9*pow(u_x + u_y, 2);
const float x17 = -x2;
const float x18 = x15 + 6*x5 + 2;


In [32]:
for i, eq in enumerate(eqs[1]):
    print("f_next_%d = %s;" % (i, ccode(eq)))

f_next_0 = f_curr_0 - x11*(72*f_curr_0 + x0*(-x1 + x9 - 9*pow(-u_x + u_y, 2)));
f_next_1 = f_curr_1 - x14*(18*f_curr_1 - x0*(x12 + x13));
f_next_2 = f_curr_2 - x11*(72*f_curr_2 - x0*(x13 + x15 + x16 + x2));
f_next_3 = f_curr_3 - x14*(18*f_curr_3 - x0*(x17 + x18));
f_next_4 = f_curr_4 - 1.0/9.0*x10*(9*f_curr_4 + 2*x0*x8);
f_next_5 = f_curr_5 - x14*(18*f_curr_5 - x0*(x18 + x2));
f_next_6 = f_curr_6 - x11*(72*f_curr_6 + x0*(x1 - x16 + x9));
f_next_7 = f_curr_7 - x14*(18*f_curr_7 + x0*(x1 - x12 + x7));
f_next_8 = f_curr_8 - x11*(72*f_curr_8 + x0*(x1 + x17 + x8 - 9*pow(u_x - u_y, 2)));


In [49]:
ueqs = cse([
    sum([ (c_i*f_curr[i])[0] for i, c_i in enumerate(c) ]) / rho,
    sum([ (c_i*f_curr[i])[1] for i, c_i in enumerate(c) ]) / rho
], optimizations='basic')

In [53]:
for eq in ueqs[0]:
    print("const float u%s = %s;" % (eq[0], ccode(eq[1])))

const float ux0 = f_curr_3 + f_curr_6;
const float ux1 = f_curr_1 + f_curr_2;
const float ux2 = 1.0/(f_curr_0 + f_curr_4 + f_curr_5 + f_curr_7 + f_curr_8 + x0 + x1);
const float ux3 = f_curr_0 - f_curr_8;


In [54]:
for i, eq in enumerate(ueqs[1]):
    print("u_%d = %s;" % (i, ccode(eq)))

u_0 = -x2*(-f_curr_2 - f_curr_5 + x0 + x3);
u_1 = x2*(-f_curr_6 - f_curr_7 + x1 + x3);
